# Constitution du dataset à partir des données exportées de CEGID expert

Dossiers exportés :
- Missions du cabinet sur 2018
- Facturation 2018
- Entrées et sorties des clients

## Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import re
from scipy import stats

## Lecture de l'ensemble des missions du cabinet sur 2018
1 ligne = 1 activité sur 1 mission réalisée par 1 assistant pour le compte d'un client

In [2]:
missions = pd.read_csv("./Data/missions_2018.txt", sep="\t", decimal='.')

In [3]:
missions.head(10)

,Code Mission,Mission,Exercice,Client,Raison sociale,DEPART CLIENT,Assistant,ASSOCIES/EC,Article,Libellé,Qté activité,Prix de vente,Total de vente,Département,SITE,Unnamed: 15
0,AJUR2018008903900,JUR,2018,07523,COPIE CREA,NaN,666,SLE,069,DIVERS JURIDIQUE,"0,50","56,00","28,00",JUR,OLIVET,NaN
1,AJUR2018008903900,JUR,2018,07523,COPIE CREA,NaN,666,SLE,069,DIVERS JURIDIQUE,"0,25","56,00","14,00",JUR,OLIVET,NaN
2,ACOM2018009385300,COM,2018,40064,E D S,NaN,556,DAC,020,FICHE DE PAIE,"0,50","64,00","32,00",SOCSJ,ANGERVILLE,NaN
3,ACOM2018009385300,COM,2018,40064,E D S,NaN,556,DAC,020,FICHE DE PAIE,"0,25","64,00","16,00",SOCSJ,ANGERVILLE,NaN
4,ACOM2018009385300,COM,2018,40064,E D S,NaN,556,DAC,021,DECLARATIONS MENSUELLES / TRIMESTRIELLES,"0,50","64,00","32,00",SOCSJ,ANGERVILLE,NaN
5,ACOM2018009114400,COM,2018,06072,LES JARDINS DE FLO,4.0,508,DAC,010,SAISIE ET CENTRALISATION,"1,00","56,00","56,00",SOCSJ,MEUNG SUR LOIRE,NaN
6,AJUR2018010737000,JUR,2018,05633,JANE A,NaN,335,SLE,060,SECRETARIAT JURIDIQUE ANNUEL,"1,00","56,00","56,00",JUR,ORLEANS,NaN
7,AJUR2018010737000,JUR,2018,05633,JANE A,NaN,335,SLE,060,SECRETARIAT JURIDIQUE ANNUEL,"0,50","56,00","28,00",JUR,ORLEANS,NaN
8,AJUR2018010737200,JUR,2018,05947,PORTE D AMONT,NaN,335,DAC,060,SECRETARIAT JURIDIQUE ANNUEL,"0,25","56,00","14,00",JUR,MEUNG SUR LOIRE,NaN
9,AJUR2018009547900,JUR,2018,07870,RESIL'IENCE,NaN,335,FRL,069,DIVERS JURIDIQUE,"0,75","56,00","42,00",JUR,GIEN,NaN


In [4]:
#Le dataset des missions 2018 contient 256 618 entrées et 16 colonnes
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256618 entries, 0 to 256617
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Code Mission     256618 non-null  object 
 1   Mission          256618 non-null  object 
 2   Exercice         256618 non-null  int64  
 3   Client           256618 non-null  object 
 4   Raison sociale   256618 non-null  object 
 5   DEPART CLIENT    46731 non-null   float64
 6   Assistant        256618 non-null  object 
 7   ASSOCIES/EC      256618 non-null  object 
 8   Article          256618 non-null  object 
 9   Libellé          256618 non-null  object 
 10  Qté activité     256618 non-null  object 
 11  Prix de vente    256618 non-null  object 
 12  Total  de vente  256618 non-null  object 
 13  Département      242750 non-null  object 
 14  SITE             256618 non-null  object 
 15  Unnamed: 15      0 non-null       float64
dtypes: float64(2), int64(1), object(13)
me

In [5]:
missions.columns

Index(['Code Mission', 'Mission', 'Exercice', 'Client', 'Raison sociale',
       'DEPART CLIENT', 'Assistant', 'ASSOCIES/EC', 'Article', 'Libellé',
       'Qté activité', 'Prix de vente', 'Total  de vente', 'Département',
       'SITE', 'Unnamed: 15'],
      dtype='object')

In [6]:
#Supprimer la dernière colonne inutile
missions.drop(columns=['Département','Unnamed: 15'], inplace=True)

In [7]:
missions.columns = ['code_mission','mission','exercice','code_client',
                    'client','depart_client','code_assistant','manager','code_article',
                    'libelle_article','temps','cout_horaire','valorisation_temps','site']

In [8]:
missions.dtypes

code_mission           object
mission                object
exercice                int64
code_client            object
client                 object
depart_client         float64
code_assistant         object
manager                object
code_article           object
libelle_article        object
temps                  object
cout_horaire           object
valorisation_temps     object
site                   object
dtype: object

In [9]:
missions.isna().sum()

code_mission               0
mission                    0
exercice                   0
code_client                0
client                     0
depart_client         209887
code_assistant             0
manager                    0
code_article               0
libelle_article            0
temps                      0
cout_horaire               0
valorisation_temps         0
site                       0
dtype: int64

## Modifier les colonnes catégorielles en float

In [10]:
#Remplacer les virgules par des points et espaces pour créer un nombre décimal
missions['temps'] = [x.replace(',', '.') for x in missions['temps']]
missions['temps'] = missions['temps'].astype(float)

In [11]:
missions['cout_horaire'] = [x.replace(',', '.') for x in missions['cout_horaire']]
missions['cout_horaire'] = [re.sub('\s','',x) for x in missions['cout_horaire']]
missions['cout_horaire'] = pd.to_numeric(missions['cout_horaire'])

In [12]:
missions['valorisation_temps'] = [x.replace(',', '.') for x in missions['valorisation_temps']]
missions['valorisation_temps'] = [re.sub('\s','',x) for x in missions['valorisation_temps']]
missions['valorisation_temps'] = pd.to_numeric(missions['valorisation_temps'])

In [13]:
missions

,code_mission,mission,exercice,code_client,client,depart_client,code_assistant,manager,code_article,libelle_article,temps,cout_horaire,valorisation_temps,site
0,AJUR2018008903900,JUR,2018,07523,COPIE CREA,NaN,666,SLE,069,DIVERS JURIDIQUE,0.50,56.0,28.00,OLIVET
1,AJUR2018008903900,JUR,2018,07523,COPIE CREA,NaN,666,SLE,069,DIVERS JURIDIQUE,0.25,56.0,14.00,OLIVET
2,ACOM2018009385300,COM,2018,40064,E D S,NaN,556,DAC,020,FICHE DE PAIE,0.50,64.0,32.00,ANGERVILLE
3,ACOM2018009385300,COM,2018,40064,E D S,NaN,556,DAC,020,FICHE DE PAIE,0.25,64.0,16.00,ANGERVILLE
4,ACOM2018009385300,COM,2018,40064,E D S,NaN,556,DAC,021,DECLARATIONS MENSUELLES / TRIMESTRIELLES,0.50,64.0,32.00,ANGERVILLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256613,ACOM2018008752700,COM,2018,06183,JAWX PROCESS COMPANY,NaN,914,DAC,010,SAISIE ET CENTRALISATION,0.50,74.0,37.00,MEUNG SUR LOIRE
256614,AJUR2018010829200,JUR,2018,06702,AVENIR R ET R,NaN,63P,SLE,060,SECRETARIAT JURIDIQUE ANNUEL,1.00,76.5,76.50,OLIVET
256615,AGEJ2018012125600,GEJ,2018,04873,AJC TAXI,NaN,340,SLE,069,DIVERS JURIDIQUE,0.25,71.0,17.75,OLIVET
256616,ACOM2018009347400,COM,2018,06101,AUZIER,NaN,656,DAC,010,SAISIE ET CENTRALISATION,0.50,71.0,35.50,MEUNG SUR LOIRE


## Agrégation du dataframe sur le code_mission pour sommer le temps et la valorisation du temps

Il faut garder une version non aggrégée pour avoir :
- les différents codes assistants par manager
- Les différents sites
- Les différents code_article et libellé articles (activités réalisées)

In [14]:
missions.code_mission.values

array(['AJUR2018008903900', 'AJUR2018008903900', 'ACOM2018009385300', ...,
       'AGEJ2018012125600', 'ACOM2018009347400', 'AGEJ2018011988500'],
      dtype=object)

In [15]:
missions_agg = missions.groupby(['code_mission']).agg({'mission':[lambda x:x.value_counts().index[0]],
                                                        'exercice':[lambda x:x.value_counts().index[0]],
                                                        'code_client': [lambda x:x.value_counts().index[0]],
                                                        'code_assistant': ['nunique'],
                                                        'manager': [lambda x:x.value_counts().index[0]],
                                                        'code_article': [lambda x: list(x.unique())],
                                                        'libelle_article': [lambda x: list(x.unique())],
                                                        'temps': ['sum'],
                                                        'valorisation_temps': ['sum'],
                                                        'site': [lambda x:x.value_counts().index[0]]})

In [16]:
missions_agg.head()

,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site
,<lambda>,<lambda>,<lambda>,nunique,<lambda>,<lambda>,<lambda>,sum,sum,<lambda>
code_mission,,,,,,,,,,
AAGS2018011606500,AGS,2018,06924,5,JOP,"[001, 029]","[RENSEIGNEMENTS CLIENT, AUTRES TRAVAUX EN SOCIAL]",26.25,2787.75,PITHIVIERS
AAGS2018011606600,AGS,2018,06565,4,FRL,"[029, 001]","[AUTRES TRAVAUX EN SOCIAL, RENSEIGNEMENTS CLIENT]",3.50,335.25,GIEN
ACAB2018010775200,CAB,2018,08937,6,CIT,"[079, 078, 098, 0711, 081, 089, 077, 086]","[ORGANISATION INTERNE, FACTURATION ET GESTION ...",191.50,13273.25,DOSSIERS INTERNES
ACAB2018010775300,CAB,2018,08949,2,CIT,"[081, 079]","[COMPTABILITE GENERALE CABINET, ORGANISATION I...",60.75,9392.00,DOSSIERS INTERNES
ACAB2018010775400,CAB,2018,08952,7,CIT,"[0792, 076, 079, 078, 085, 081, 091, 092, 088,...","[RESSOURCES HUMAINES, GESTION DES TEMPS, ORGAN...",332.25,45174.75,DOSSIERS INTERNES


In [17]:
missions_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9021 entries, AAGS2018011606500 to ATB 2018012107700
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (mission, <lambda>)          9021 non-null   object 
 1   (exercice, <lambda>)         9021 non-null   int64  
 2   (code_client, <lambda>)      9021 non-null   object 
 3   (code_assistant, nunique)    9021 non-null   int64  
 4   (manager, <lambda>)          9021 non-null   object 
 5   (code_article, <lambda>)     9021 non-null   object 
 6   (libelle_article, <lambda>)  9021 non-null   object 
 7   (temps, sum)                 9021 non-null   float64
 8   (valorisation_temps, sum)    9021 non-null   float64
 9   (site, <lambda>)             9021 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 775.2+ KB


## Lecture du dossier facturation sur 2018
Quel coût facturé au client ?

In [39]:
facturation = pd.read_csv("./Data/facturation_20182.txt", sep="\t")

In [40]:
facturation.head()

,Code mission,Client,Exercice,Mission,Total HT,Exercice.1,Date,Nature,N° Pièce,Article,Qté facturée,Prix unitaire,Numéro d'ordre,Désignation,Raison sociale,Unnamed: 15
0,ACOM2018012942600,A1643,2018,COM,"400,00",2018,06/11/2019,FAC,51534,ACOMPTE,1,"400,00",1,Bilan au 31/7/2018,MMH TAXI,NaN
1,AJUR2018012942700,A1643,2018,JUR,"200,00",2018,06/11/2019,FAC,51534,ACOMPTE,1,"200,00",3,Juridique 31/07/2018,MMH TAXI,NaN
2,AANT2018013216000,A1696,2018,ANT,"-450,00",2018,31/01/2020,FAC,52223,ACOMPTE,1,"-450,00",3,Annulation assurance contrôle fiscal,GROUPE GARNIER IMMOBILIER,NaN
3,AANT2018013216000,A1696,2018,ANT,"-4 080,00",2018,31/01/2020,FAC,52223,ACOMPTE,1,"-4 080,00",4,Geste commercial antériorité,GROUPE GARNIER IMMOBILIER,NaN
4,AANT2018013216000,A1696,2018,ANT,"-710,00",2018,31/01/2020,FAC,52223,ACOMPTE,1,"-710,00",5,Avoir conseils fiscaux,GROUPE GARNIER IMMOBILIER,NaN


In [41]:
facturation.dtypes

Code mission       object
Client             object
Exercice            int64
Mission            object
Total HT           object
Exercice.1          int64
Date               object
Nature             object
N° Pièce            int64
Article            object
Qté facturée        int64
Prix unitaire      object
Numéro d'ordre      int64
Désignation        object
Raison sociale     object
Unnamed: 15       float64
dtype: object

In [42]:
facturation[facturation['Exercice'].isna()]

,Code mission,Client,Exercice,Mission,Total HT,Exercice.1,Date,Nature,N° Pièce,Article,Qté facturée,Prix unitaire,Numéro d'ordre,Désignation,Raison sociale,Unnamed: 15


In [43]:
facturation.isnull().sum()

Code mission          0
Client                0
Exercice              0
Mission               0
Total HT              0
Exercice.1            0
Date                  0
Nature                0
N° Pièce              0
Article               0
Qté facturée          0
Prix unitaire         0
Numéro d'ordre        0
Désignation           6
Raison sociale        0
Unnamed: 15       38201
dtype: int64

In [44]:
# Remplacer les valeurs catégorielles par des floats
facturation['Total HT'] = [str(x).replace(',', '.') for x in facturation['Total HT']]
facturation['Total HT'] = [re.sub('\s','',x) for x in facturation['Total HT']]
facturation['Total HT'] = pd.to_numeric(facturation['Total HT'])

In [45]:
facturation['Total HT'].isnull().sum()

0

### Remplir les valeurs NaN par 0 dans le Total HT de la facturation 

In [46]:
facturation.fillna(0, inplace=True)

In [47]:
facturation['Qté facturée'].value_counts().head(10)

1    25922
2     4003
3     1974
4     1278
5      925
6      721
7      489
8      442
0      432
9      296
Name: Qté facturée, dtype: int64

## Agrégation du dataset facturation sur le code_mission pour sommer le total HT facturé au client sur une mission particulière

In [48]:
fact_agg = facturation.groupby(['Code mission']).agg({'Total HT':['sum']})

In [49]:
fact_agg.head()

,Total HT
,sum
Code mission,
AAGS2018011606500,3000.00
AAGS2018011606600,800.00
AANT2018010597600,10869.57
AANT2018010624400,4013.38
AANT2018010762300,4775.13


Il existe plus de missions dans facturation que de missions dans le dataset mission ?

## Concaténation du dataset facturation sur l'ensemble des missions

In [50]:
data = pd.concat([missions_agg, fact_agg], axis=1)

In [51]:
data.columns = ['mission','exercice','code_client','code_assistant','manager','code_article',
                'libelle_article','temps','valorisation_temps','site','facturation']

In [52]:
#306 factures à 0 euro
data.isna().sum()

mission               157
exercice              157
code_client           157
code_assistant        157
manager               157
code_article          157
libelle_article       157
temps                 157
valorisation_temps    157
site                  157
facturation           306
dtype: int64

In [53]:
len(data.index)

9178

In [54]:
#Remplacer NaN par 0 euro
data['facturation'].fillna(0, inplace=True)

In [56]:
#Le reste des valeurs nulles correspondent au facture sur des codes_missions qui n'existent pas dans le dataset missions
data.isna().sum()

mission               157
exercice              157
code_client           157
code_assistant        157
manager               157
code_article          157
libelle_article       157
temps                 157
valorisation_temps    157
site                  157
facturation             0
dtype: int64

In [57]:
list_index = data[data['code_client'].isna()].index
len(list_index)

157

In [58]:
data.drop(list_index, axis=0, inplace=True)

In [59]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9021 entries, AAGS2018011606500 to ATB 2018012107700
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mission             9021 non-null   object 
 1   exercice            9021 non-null   float64
 2   code_client         9021 non-null   object 
 3   code_assistant      9021 non-null   float64
 4   manager             9021 non-null   object 
 5   code_article        9021 non-null   object 
 6   libelle_article     9021 non-null   object 
 7   temps               9021 non-null   float64
 8   valorisation_temps  9021 non-null   float64
 9   site                9021 non-null   object 
 10  facturation         9021 non-null   float64
dtypes: float64(5), object(6)
memory usage: 845.7+ KB


In [60]:
data.head()

,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation
AAGS2018011606500,AGS,2018.0,06924,5.0,JOP,"[001, 029]","[RENSEIGNEMENTS CLIENT, AUTRES TRAVAUX EN SOCIAL]",26.25,2787.75,PITHIVIERS,3000.0
AAGS2018011606600,AGS,2018.0,06565,4.0,FRL,"[029, 001]","[AUTRES TRAVAUX EN SOCIAL, RENSEIGNEMENTS CLIENT]",3.50,335.25,GIEN,800.0
ACAB2018010775200,CAB,2018.0,08937,6.0,CIT,"[079, 078, 098, 0711, 081, 089, 077, 086]","[ORGANISATION INTERNE, FACTURATION ET GESTION ...",191.50,13273.25,DOSSIERS INTERNES,40039.0
ACAB2018010775300,CAB,2018.0,08949,2.0,CIT,"[081, 079]","[COMPTABILITE GENERALE CABINET, ORGANISATION I...",60.75,9392.00,DOSSIERS INTERNES,0.0
ACAB2018010775400,CAB,2018.0,08952,7.0,CIT,"[0792, 076, 079, 078, 085, 081, 091, 092, 088,...","[RESSOURCES HUMAINES, GESTION DES TEMPS, ORGAN...",332.25,45174.75,DOSSIERS INTERNES,5655.6


## Ajouter les entrées et sorties des clients MCCM

In [61]:
clients = pd.read_csv("./Data/client_dates.csv", sep=";")

In [62]:
clients_formes = pd.read_csv("./Data/client-naf-forme.csv", sep=";")

In [63]:
clients.head()

,Code,Date création,Départ cabinet
0,30,07/11/1995,0
1,44,31/10/1995,0
2,441,30/10/1995,0
3,455,07/11/1995,0
4,549,09/11/1995,0


In [64]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11281 entries, 0 to 11280
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Code            11281 non-null  object
 1   Date création   11277 non-null  object
 2   Départ cabinet  11281 non-null  object
dtypes: object(3)
memory usage: 264.5+ KB


In [65]:
clients.dtypes

Code              object
Date création     object
Départ cabinet    object
dtype: object

In [66]:
def entree_sortie(col):
    
    for date in str(col):
        if '2018' in str(col):
            return 1
        else:
            return 0

In [67]:
#Test de la fonction
entree_sortie("20/02/2018")

1

In [68]:
clients['entrée_clt'] = clients['Date création'].apply(entree_sortie)

In [69]:
clients['sortie_clt'] = clients['Départ cabinet'].apply(entree_sortie)

In [70]:
clients.head(50)

,Code,Date création,Départ cabinet,entrée_clt,sortie_clt
0,30,07/11/1995,0,0,0
1,44,31/10/1995,0,0,0
2,441,30/10/1995,0,0,0
3,455,07/11/1995,0,0,0
4,549,09/11/1995,0,0,0
5,569,06/11/1995,0,0,0
6,579,06/11/1995,31/03/2020,0,0
7,584,10/11/1995,0,0,0
8,613,16/11/1995,0,0,0
9,632,30/10/1995,0,0,0


In [71]:
clients.columns

Index(['Code', 'Date création', 'Départ cabinet', 'entrée_clt', 'sortie_clt'], dtype='object')

In [72]:
clients.columns = ['code_client', 'ENTREE_CLIENT', 'SORTIE_CLIENT', 'entrée_clt', 'sortie_clt']

In [73]:
clients.code_client.value_counts()

A1779    1
3359     1
80200    1
2824     1
4482     1
        ..
5323     1
5638     1
77240    1
A3721    1
A0865    1
Name: code_client, Length: 11281, dtype: int64

In [74]:
clients_formes.head()

,Code,Code NAF,Forme juridique
0,9,Divers,SA
1,25,Autre imprimerie (labeur),SARL
2,26,Édition de revues et périodiques,SA
3,30,Fabrication de parquets assemblés,SAS
4,36,Commerce de détail d'habillement en magasin sp...,SARL


## Concaténer les entrées et les sorties au dataset selon le code_client avec la fonction merge()

In [75]:
clients.code_client

0           30
1           44
2          441
3          455
4          549
         ...  
11276    P7129
11277    P7503
11278    P7540
11279    P7673
11280    P7911
Name: code_client, Length: 11281, dtype: object

In [76]:
len(clients.code_client[0])

2

In [77]:
def suppr_0(s):
    return s.lstrip("0")

In [78]:
#Test de la suppression de 0 au début des identifiants
suppr_0("06565")

'6565'

In [79]:
data.code_client = data.code_client.apply(suppr_0)

In [80]:
data.head()

,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation
AAGS2018011606500,AGS,2018.0,6924,5.0,JOP,"[001, 029]","[RENSEIGNEMENTS CLIENT, AUTRES TRAVAUX EN SOCIAL]",26.25,2787.75,PITHIVIERS,3000.0
AAGS2018011606600,AGS,2018.0,6565,4.0,FRL,"[029, 001]","[AUTRES TRAVAUX EN SOCIAL, RENSEIGNEMENTS CLIENT]",3.50,335.25,GIEN,800.0
ACAB2018010775200,CAB,2018.0,8937,6.0,CIT,"[079, 078, 098, 0711, 081, 089, 077, 086]","[ORGANISATION INTERNE, FACTURATION ET GESTION ...",191.50,13273.25,DOSSIERS INTERNES,40039.0
ACAB2018010775300,CAB,2018.0,8949,2.0,CIT,"[081, 079]","[COMPTABILITE GENERALE CABINET, ORGANISATION I...",60.75,9392.00,DOSSIERS INTERNES,0.0
ACAB2018010775400,CAB,2018.0,8952,7.0,CIT,"[0792, 076, 079, 078, 085, 081, 091, 092, 088,...","[RESSOURCES HUMAINES, GESTION DES TEMPS, ORGAN...",332.25,45174.75,DOSSIERS INTERNES,5655.6


In [81]:
data[data['code_client'] == '6087SAS'].index

Index(['ACOM2018009347200', 'AGES2018011623000', 'ASOP2018011541300'], dtype='object')

In [82]:
data['code_client'].replace(['6087SAS'], '06087SAS', inplace=True)

In [83]:
data.reset_index(inplace=True)

In [84]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9021 entries, 0 to 9020
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               9021 non-null   object 
 1   mission             9021 non-null   object 
 2   exercice            9021 non-null   float64
 3   code_client         9021 non-null   object 
 4   code_assistant      9021 non-null   float64
 5   manager             9021 non-null   object 
 6   code_article        9021 non-null   object 
 7   libelle_article     9021 non-null   object 
 8   temps               9021 non-null   float64
 9   valorisation_temps  9021 non-null   float64
 10  site                9021 non-null   object 
 11  facturation         9021 non-null   float64
dtypes: float64(5), object(7)
memory usage: 845.8+ KB


In [85]:
data_2018 = pd.merge(data, clients[['code_client','entrée_clt', 'sortie_clt']], how='left', on="code_client")

In [86]:
data_2018.head()

,index,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation,entrée_clt,sortie_clt
0,AAGS2018011606500,AGS,2018.0,6924,5.0,JOP,"[001, 029]","[RENSEIGNEMENTS CLIENT, AUTRES TRAVAUX EN SOCIAL]",26.25,2787.75,PITHIVIERS,3000.0,0,0
1,AAGS2018011606600,AGS,2018.0,6565,4.0,FRL,"[029, 001]","[AUTRES TRAVAUX EN SOCIAL, RENSEIGNEMENTS CLIENT]",3.50,335.25,GIEN,800.0,0,0
2,ACAB2018010775200,CAB,2018.0,8937,6.0,CIT,"[079, 078, 098, 0711, 081, 089, 077, 086]","[ORGANISATION INTERNE, FACTURATION ET GESTION ...",191.50,13273.25,DOSSIERS INTERNES,40039.0,0,0
3,ACAB2018010775300,CAB,2018.0,8949,2.0,CIT,"[081, 079]","[COMPTABILITE GENERALE CABINET, ORGANISATION I...",60.75,9392.00,DOSSIERS INTERNES,0.0,0,0
4,ACAB2018010775400,CAB,2018.0,8952,7.0,CIT,"[0792, 076, 079, 078, 085, 081, 091, 092, 088,...","[RESSOURCES HUMAINES, GESTION DES TEMPS, ORGAN...",332.25,45174.75,DOSSIERS INTERNES,5655.6,0,0


In [88]:
clients_formes.head()

,Code,Code NAF,Forme juridique
0,9,Divers,SA
1,25,Autre imprimerie (labeur),SARL
2,26,Édition de revues et périodiques,SA
3,30,Fabrication de parquets assemblés,SAS
4,36,Commerce de détail d'habillement en magasin sp...,SARL


In [69]:
clients_formes.columns

Index(['Code', 'Code NAF', 'Forme juridique'], dtype='object')

In [89]:
clients_formes.columns = ['code_client', 'Code NAF', 'Forme juridique']

In [90]:
data_final_2018 = pd.merge(data_2018, clients_formes[['code_client', 'Code NAF','Forme juridique']], how='left', on="code_client")

In [91]:
#Vérification des valeurs nulles
data_final_2018.isna().sum()

index                 0
mission               0
exercice              0
code_client           0
code_assistant        0
manager               0
code_article          0
libelle_article       0
temps                 0
valorisation_temps    0
site                  0
facturation           0
entrée_clt            0
sortie_clt            0
Code NAF              0
Forme juridique       0
dtype: int64

In [92]:
data_final_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9021 entries, 0 to 9020
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               9021 non-null   object 
 1   mission             9021 non-null   object 
 2   exercice            9021 non-null   float64
 3   code_client         9021 non-null   object 
 4   code_assistant      9021 non-null   float64
 5   manager             9021 non-null   object 
 6   code_article        9021 non-null   object 
 7   libelle_article     9021 non-null   object 
 8   temps               9021 non-null   float64
 9   valorisation_temps  9021 non-null   float64
 10  site                9021 non-null   object 
 11  facturation         9021 non-null   float64
 12  entrée_clt          9021 non-null   int64  
 13  sortie_clt          9021 non-null   int64  
 14  Code NAF            9021 non-null   object 
 15  Forme juridique     9021 non-null   object 
dtypes: flo

In [93]:
#Vérification de combien de lignes par client
data_final_2018.code_client.value_counts()

713      11
3020      9
3023      8
7195      8
3598      8
         ..
A0686     1
5269      1
A0099     1
A0499     1
6701      1
Name: code_client, Length: 3966, dtype: int64

In [94]:
data_final_2018[data_final_2018['code_client']=='88912']

,index,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation,entrée_clt,sortie_clt,Code NAF,Forme juridique
4946,AGEP2018011664400,GEP,2018.0,88912,1.0,ALD,[029],[AUTRES TRAVAUX EN SOCIAL],0.75,51.0,OLIVET,63.75,0,0,Travaux de peinture et vitrerie,SARL
8772,ASOP2018011582400,SOP,2018.0,88912,5.0,ALD,"[029, F0200, F0216, 021, 020, 022]","[AUTRES TRAVAUX EN SOCIAL, BULLETIN DE PAIE, B...",42.15,1760.1,OLIVET,673.00,0,0,Travaux de peinture et vitrerie,SARL


In [95]:
data_final_2018.dtypes

index                  object
mission                object
exercice              float64
code_client            object
code_assistant        float64
manager                object
code_article           object
libelle_article        object
temps                 float64
valorisation_temps    float64
site                   object
facturation           float64
entrée_clt              int64
sortie_clt              int64
Code NAF               object
Forme juridique        object
dtype: object

In [98]:
data_final_2018[data_final_2018['type_mission'] == "SST"].index

Int64Index([9014, 9015], dtype='int64')

In [99]:
data_final_2018.drop([9014, 9015], axis=0, inplace=True)

## Exportation du dataset 2018

In [97]:
data_final_2018.columns = ['code_mission', 'type_mission', 'exercice', 'code_client', 'total_assistant',
                           'manager', 'code_article', 'libelle_article', 'temps',
                           'valorisation_temps', 'site', 'facturation', 'entrée_clt', 'sortie_clt', 'secteur', 'forme']

In [100]:
data_final_2018.head()

,code_mission,type_mission,exercice,code_client,total_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation,entrée_clt,sortie_clt,secteur,forme
0,AAGS2018011606500,AGS,2018.0,6924,5.0,JOP,"[001, 029]","[RENSEIGNEMENTS CLIENT, AUTRES TRAVAUX EN SOCIAL]",26.25,2787.75,PITHIVIERS,3000.0,0,0,Fabrication de meubles de bureau et de magasin,SAS
1,AAGS2018011606600,AGS,2018.0,6565,4.0,FRL,"[029, 001]","[AUTRES TRAVAUX EN SOCIAL, RENSEIGNEMENTS CLIENT]",3.50,335.25,GIEN,800.0,0,0,Activités d'architecture,EURL
2,ACAB2018010775200,CAB,2018.0,8937,6.0,CIT,"[079, 078, 098, 0711, 081, 089, 077, 086]","[ORGANISATION INTERNE, FACTURATION ET GESTION ...",191.50,13273.25,DOSSIERS INTERNES,40039.0,0,0,Conseil en systèmes et logiciels informatiques,EURL
3,ACAB2018010775300,CAB,2018.0,8949,2.0,CIT,"[081, 079]","[COMPTABILITE GENERALE CABINET, ORGANISATION I...",60.75,9392.00,DOSSIERS INTERNES,0.0,0,0,Activités comptables,SASU
4,ACAB2018010775400,CAB,2018.0,8952,7.0,CIT,"[0792, 076, 079, 078, 085, 081, 091, 092, 088,...","[RESSOURCES HUMAINES, GESTION DES TEMPS, ORGAN...",332.25,45174.75,DOSSIERS INTERNES,5655.6,0,0,Activités comptables,SAS


In [101]:
data_final_2018.to_csv("./Data/dataset-2018.csv", index=False)